# Practical Sheet 1: Hill Climbing and Simulated Annealing 

## Imports:

In [1]:
import numpy as np
import random
import math
import time

## Q1, Q2, Q3) Shared initialization blocks:

In [2]:
x = np.arange(-10, 10.01, 0.01)
y = np.arange(-10, 10.01, 0.01)
z = np.arange(-10, 10.01, 0.01)

def sinc(x):
    return np.where(x == 0, 1.0, np.sin(x) / x)

def clip(val, lower=-10, upper=10):
    return np.clip(val, lower, upper)

def f1(x):
  return sinc(x - 3)

def f2(x, y):
  return sinc(x + y + 2)

def f3(x, y, z):
  return sinc(x**2 + y**2 + z**2 - 1)

## Q1) Answer:

In [8]:
f1_tuple = tuple((x[i], f1(x[i])) for i in range(len(x)))
f2_tuple = tuple((x[i], y[j], f2(x[i], y[j])) for i, j in zip(range(len(x)), range(len(y))))
f3_tuple = tuple((x[i], y[j], z[k], f3(x[i], y[j], z[k])) for i, j, k in zip(range(len(x)), range(len(y)), range(len(z))))

print(f'Variable x = {max(f1_tuple, key=lambda t: t[1])[0]:.5f} , f1 Value = {max(f1_tuple, key=lambda t: t[1])[1]:.5f}')
print(f'Variable x = {max(f2_tuple, key=lambda t: t[2])[0]:.5f} , Variable y = {max(f2_tuple, key=lambda t: t[2])[1]:.5f} , f2 Value = {max(f2_tuple, key=lambda t: t[2])[2]:.5f}')
print(f'Variable x = {max(f3_tuple, key=lambda t: t[3])[0]:.5f} , Variable y = {max(f3_tuple, key=lambda t: t[3])[1]:.5f} , Variable z = {max(f3_tuple, key=lambda t: t[3])[2]:.5f} , f3 Value = {max(f3_tuple, key=lambda t: t[3])[3]:.5f}')

Variable x = 3.00000 , f1 Value = 1.00000
Variable x = -1.00000 , Variable y = -1.00000 , f2 Value = 1.00000
Variable x = 0.58000 , Variable y = 0.58000 , Variable z = 0.58000 , f3 Value = 0.99999


## Q2) Answer:

In [10]:
def randomMutationHillClimbing3D(f, x, y, z, steps=100):
  best = f(x[0], y[0], z[0])
  best_x, best_y, best_z = x[0], y[0], z[0]
  for _ in range(steps):
    x_new = np.random.choice(x)
    y_new = np.random.choice(y)
    z_new = np.random.choice(z)
    new_value = f(x_new, y_new, z_new)
    if new_value > best:
      best = new_value
      best_x, best_y, best_z = x_new, y_new, z_new
  return best_x, best_y, best_z, best

def randomMutationHillClimbing2D(f, x, y, steps=100):
  best = f(x[0], y[0])
  best_x, best_y = x[0], y[0]
  for _ in range(steps):
    x_new = np.random.choice(x)
    y_new = np.random.choice(y)
    new_value = f(x_new, y_new)
    if new_value > best:
      best = new_value
      best_x, best_y = x_new, y_new
  return best_x, best_y, best

def randomMutationHillClimbing1D(f, x, steps=100):
  best = f(x[0])
  best_x = x[0]
  for _ in range(steps):
    x_new = np.random.choice(x)
    new_value = f(x_new)
    if new_value > best:
      best = new_value
      best_x= x_new
  return best_x, best

resultsF1 = randomMutationHillClimbing1D(f1, x, steps=100)
resultsF2 = randomMutationHillClimbing2D(f2, x, y, steps=100)
resultsF3 = randomMutationHillClimbing3D(f3, x, y, z, steps=100)

print(f'Best x: {resultsF1[0]:.5f}, Best value: {resultsF1[1]:.5f}')
print(f'Best x: {resultsF2[0]:.5f}, Best y: {resultsF2[1]:.5f}, Best value: {resultsF2[2]:.5f}')
print(f'Best x: {resultsF3[0]:.5f}, Best y: {resultsF3[1]:.5f}, Best z: {resultsF3[2]:.5f}, Best value: {resultsF3[3]:.5f}')

Best x: 3.20000, Best value: 0.99335
Best x: 5.83000, Best y: -8.07000, Best value: 0.99043
Best x: -0.41000, Best y: -0.59000, Best z: 0.57000, Best value: 0.99580


## Q3) Answer:

In [6]:
def nextAscentHillClimbing1D(f, steps=1000, step_size=0.01):
    x = np.random.uniform(-10, 10)
    best = f(x)
    for _ in range(steps):
        x_new = clip(x + np.random.uniform(-step_size, step_size))
        f_new = f(x_new)
        if f_new > best:
            x, best = x_new, f_new
        else:
            # Random restart
            x = np.random.uniform(-10, 10)
            best = f(x)
    return x, best

def nextAscentHillClimbing2D(f, steps=1000, step_size=0.01):
    x, y = np.random.uniform(-10, 10, 2)
    best = f(x, y)
    for _ in range(steps):
        x_new = clip(x + np.random.uniform(-step_size, step_size))
        y_new = clip(y + np.random.uniform(-step_size, step_size))
        f_new = f(x_new, y_new)
        if f_new > best:
            x, y, best = x_new, y_new, f_new
        else:
            x, y = np.random.uniform(-10, 10, 2)
            best = f(x, y)
    return x, y, best

def nextAscentHillClimbing3D(f, steps=1000, step_size=0.01):
    x, y, z = np.random.uniform(-10, 10, 3)
    best = f(x, y, z)
    for _ in range(steps):
        x_new = clip(x + np.random.uniform(-step_size, step_size))
        y_new = clip(y + np.random.uniform(-step_size, step_size))
        z_new = clip(z + np.random.uniform(-step_size, step_size))
        f_new = f(x_new, y_new, z_new)
        if f_new > best:
            x, y, z, best = x_new, y_new, z_new, f_new
        else:
            x, y, z = np.random.uniform(-10, 10, 3)
            best = f(x, y, z)
    return x, y, z, best

# Run
resultsF1 = nextAscentHillClimbing1D(f1)
resultsF2 = nextAscentHillClimbing2D(f2)
resultsF3 = nextAscentHillClimbing3D(f3)

print(f'[F1] Best x = {resultsF1[0]:.5f}, Value = {resultsF1[1]:.5f}')
print(f'[F2] Best x = {resultsF2[0]:.5f}, y = {resultsF2[1]:.5f}, Value = {resultsF2[2]:.5f}')
print(f'[F3] Best x = {resultsF3[0]:.5f}, y = {resultsF3[1]:.5f}, z = {resultsF3[2]:.5f}, Value = {resultsF3[3]:.5f}')

# Applying Monte Carlo Simulation on 1D,2D,3D functions:

def monteCarloNextAschentSimulation1D(f, iterations=100):
    sum_values = 0
    for _ in range(iterations):
       res = nextAscentHillClimbing1D(f, steps=1000)
       sum_values += res[1]
    return sum_values / iterations

def monteCarloNextAschentSimulation2D(f, iterations=100):
    sum_values = 0
    for _ in range(iterations):
       res = nextAscentHillClimbing2D(f, steps=1000)
       sum_values += res[2]
    return sum_values / iterations

def monteCarloNextAschentSimulation3D(f, iterations=100):
    sum_values = 0
    for _ in range(iterations):
       res = nextAscentHillClimbing3D(f, steps=1000)
       sum_values += res[3]
    return sum_values / iterations

print(f'[F1] Monte Carlo Simulation Value for next ascent = {monteCarloNextAschentSimulation1D(f1,100):.5f}')
print(f'[F2] Monte Carlo Simulation Value for next ascent = {monteCarloNextAschentSimulation2D(f2,100):.5f}')
print(f'[F3] Monte Carlo Simulation Value for next ascent = {monteCarloNextAschentSimulation3D(f3,100):.5f}')



def steepestAscentHillClimbing1D(f, steps=1000, step_size=0.01):
    x = np.random.uniform(-10, 10)
    best = f(x)
    for _ in range(steps):

        neighbors = [clip(x + np.random.uniform(-step_size, step_size)) for _ in range(10)]
        evaluations = [f(neighbor) for neighbor in neighbors]
        
        max_index = np.argmax(evaluations)
        x_new, f_new = neighbors[max_index], evaluations[max_index]
        
        if f_new > best:
            x, best = x_new, f_new
        else:
            x = np.random.uniform(-10, 10)
            best = f(x)
    return x, best

def steepestAscentHillClimbing2D(f, steps=1000, step_size=0.01):
    x, y = np.random.uniform(-10, 10, 2)
    best = f(x, y)
    for _ in range(steps):
        neighbors = [(clip(x + np.random.uniform(-step_size, step_size)),
                      clip(y + np.random.uniform(-step_size, step_size))) for _ in range(10)]
        evaluations = [f(nx, ny) for nx, ny in neighbors]
        
        max_index = np.argmax(evaluations)
        x_new, y_new = neighbors[max_index]
        f_new = evaluations[max_index]
        
        if f_new > best:
            x, y, best = x_new, y_new, f_new
        else:
            x, y = np.random.uniform(-10, 10, 2)
            best = f(x, y)
    return x, y, best

def steepestAscentHillClimbing3D(f, steps=1000, step_size=0.01):
    x, y, z = np.random.uniform(-10, 10, 3)
    best = f(x, y, z)
    for _ in range(steps):
        neighbors = [(clip(x + np.random.uniform(-step_size, step_size)),
                      clip(y + np.random.uniform(-step_size, step_size)),
                      clip(z + np.random.uniform(-step_size, step_size))) for _ in range(10)]
        evaluations = [f(nx, ny, nz) for nx, ny, nz in neighbors]
        
        max_index = np.argmax(evaluations)
        x_new, y_new, z_new = neighbors[max_index]
        f_new = evaluations[max_index]
        
        if f_new > best:
            x, y, z, best = x_new, y_new, z_new, f_new
        else:
            x, y, z = np.random.uniform(-10, 10, 3)
            best = f(x, y, z)
    return x, y, z, best

def monteCarloSimulationSteepestAscent1D(f, iterations=100):
    sum_values = 0
    for _ in range(iterations):
       res = steepestAscentHillClimbing1D(f, steps=1000)
       sum_values += res[1]
    return sum_values / iterations

def monteCarloSimulationSteepestAscent2D(f, iterations=100):
    sum_values = 0
    for _ in range(iterations):
       res = steepestAscentHillClimbing2D(f, steps=1000)
       sum_values += res[2]
    return sum_values / iterations

def monteCarloSimulationSteepestAscent3D(f, iterations=100):
    sum_values = 0
    for _ in range(iterations):
       res = steepestAscentHillClimbing3D(f, steps=1000)
       sum_values += res[3]
    return sum_values / iterations

print(f'[F1] Monte Carlo Simulation Value for steepest ascent = {monteCarloSimulationSteepestAscent1D(f1,100):.5f}')
print(f'[F2] Monte Carlo Simulation Value for steepest ascent = {monteCarloSimulationSteepestAscent2D(f2,100):.5f}')
print(f'[F3] Monte Carlo Simulation Value for steepest ascent = {monteCarloSimulationSteepestAscent3D(f3,100):.5f}')

[F1] Best x = -6.08743, Value = 0.03642
[F2] Best x = 4.86707, y = 1.27386, Value = 0.11781
[F3] Best x = 5.13197, y = -3.09037, z = 6.81436, Value = -0.00431
[F1] Monte Carlo Simulation Value for next ascent = 0.08843
[F2] Monte Carlo Simulation Value for next ascent = 0.10240
[F3] Monte Carlo Simulation Value for next ascent = 0.00274
[F1] Monte Carlo Simulation Value for steepest ascent = 0.35653
[F2] Monte Carlo Simulation Value for steepest ascent = 0.26062
[F3] Monte Carlo Simulation Value for steepest ascent = 0.05682


### Which algorithm got the better results? 
Answer: Steepest Ascent Hill got much better results as the average is closer to value of 1 than the value of the Monte Carlo simulations for the next ascent hill climbing.

That returns to the conservative behaviour of the steepest ascent hill rather than being greedy and fast like the next ascent hill.

---

### Which algorithm executes faster on average? 
Answer: Next ascent hill.

# Q4) Answer:

In [7]:

# Define the distance matrix for 8 cities
distance = [
    [0, 4, 10, math.inf, math.inf, math.inf, math.inf, 5],
    [4, 0, 11, 15, math.inf, math.inf, math.inf, math.inf],
    [10, 11, 0, 13, 3, math.inf, math.inf, 11],
    [math.inf, 15, 13, 0, 6, 5, math.inf, math.inf],
    [math.inf, math.inf, 3, 6, 0, 2, 5, math.inf],
    [math.inf, math.inf, math.inf, 5, 2, 0, 8, math.inf],
    [math.inf, math.inf, math.inf, math.inf, 5, 8, 0, 7],
    [5, math.inf, 11, math.inf, math.inf, math.inf, 7, 0]
]

# Function to calculate the total distance of a tour
def total_distance(tour, dist_matrix):
    n = len(tour)
    dist = 0
    for i in range(n-1):
        if dist_matrix[tour[i]-1][tour[i+1]-1] == math.inf:
            return math.inf
        dist += dist_matrix[tour[i]-1][tour[i+1]-1]
    if dist_matrix[tour[-1]-1][tour[0]-1] == math.inf:
        return math.inf
    dist += dist_matrix[tour[-1]-1][tour[0]-1]  # Return to start
    return dist

# Steepest ascent hill climbing (descent for minimization)
def hill_climbing(dist_matrix, max_iter=1000):
    # Generate initial tour: [1] + random permutation of [2,3,4,5,6,7,8]
    while True:
        tour = [1] + random.sample([2, 3, 4, 5, 6, 7, 8], 7)
        if total_distance(tour, dist_matrix) != math.inf:
            break
    current_distance = total_distance(tour, dist_matrix)
    
    for _ in range(max_iter):
        # Generate all neighbors by swapping any two cities
        neighbors = []
        for i in range(8):
            for j in range(i + 1, 8):
                neighbor = tour.copy()
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                if total_distance(neighbor, dist_matrix) != math.inf:
                    neighbors.append(neighbor)
        
        if not neighbors:  # No valid neighbors
            break
        
        # Find the best neighbor
        best_neighbor = min(neighbors, key=lambda t: total_distance(t, dist_matrix))
        best_distance = total_distance(best_neighbor, dist_matrix)
        
        # Move to the best neighbor if it improves the current distance
        if best_distance < current_distance:
            tour = best_neighbor
            current_distance = best_distance
        else:
            break  # Stop if no improvement is found
    
    return tour, current_distance

# Simulated annealing with linear cooling
def simulated_annealing(dist_matrix, max_iter=1000, T0=50, Tf=0.1):
    # Generate initial tour
    while True:
        tour = [1] + random.sample([2, 3, 4, 5, 6, 7, 8], 7)
        if total_distance(tour, dist_matrix) != math.inf:
            break
    current_distance = total_distance(tour, dist_matrix)
    best_tour = tour.copy()
    best_distance = current_distance
    
    for k in range(max_iter):
        # Linear cooling schedule
        T = T0 - (T0 - Tf) * (k / (max_iter - 1))
        
        # Generate a random neighbor by swapping two cities
        i, j = random.sample(range(8), 2)
        neighbor = tour.copy()
        neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        neighbor_distance = total_distance(neighbor, dist_matrix)
        
        if neighbor_distance == math.inf:
            continue  # Skip invalid neighbors
        
        # Calculate the change in distance
        delta_E = neighbor_distance - current_distance
        
        # Accept the new tour if better or with probability based on temperature
        if delta_E < 0 or random.random() < math.exp(-delta_E / T):
            tour = neighbor
            current_distance = neighbor_distance
        
        # Update the best tour if the current is better
        if current_distance < best_distance:
            best_tour = tour.copy()
            best_distance = current_distance
    
    return best_tour, best_distance

# Run Monte-Carlo trials and compare results
def run_experiments():
    num_trials = 100
    
    # Hill Climbing trials
    hc_times = []
    hc_best_distances = []
    for _ in range(num_trials):
        start_time = time.time()
        tour, dist = hill_climbing(distance)
        end_time = time.time()
        hc_times.append(end_time - start_time)
        hc_best_distances.append(dist)
    
    hc_min_distance = min(hc_best_distances)
    hc_avg_time = sum(hc_times) / num_trials
    
    # Simulated Annealing trials
    sa_times = []
    sa_best_distances = []
    for _ in range(num_trials):
        start_time = time.time()
        tour, dist = simulated_annealing(distance)
        end_time = time.time()
        sa_times.append(end_time - start_time)
        sa_best_distances.append(dist)
    
    sa_min_distance = min(sa_best_distances)
    sa_avg_time = sum(sa_times) / num_trials
    
    # Print results
    print("### Hill Climbing Results")
    print(f"Shortest path distance: {hc_min_distance}")
    print(f"Average execution time per run: {hc_avg_time:.6f} seconds")
    print("\n### Simulated Annealing Results")
    print(f"Shortest path distance: {sa_min_distance}")
    print(f"Average execution time per run: {sa_avg_time:.6f} seconds")
    print("\n### Comparison")
    if hc_min_distance < sa_min_distance:
        print("Hill Climbing got the better result (shorter distance).")
    elif sa_min_distance < hc_min_distance:
        print("Simulated Annealing got the better result (shorter distance).")
    else:
        print("Both algorithms achieved the same shortest distance.")
    if hc_avg_time < sa_avg_time:
        print("Hill Climbing executed faster on average.")
    else:
        print("Simulated Annealing executed faster on average.")

# Execute the experiments
if __name__ == "__main__":
    run_experiments()

### Hill Climbing Results
Shortest path distance: 49
Average execution time per run: 0.001265 seconds

### Simulated Annealing Results
Shortest path distance: 49
Average execution time per run: 0.004921 seconds

### Comparison
Both algorithms achieved the same shortest distance.
Hill Climbing executed faster on average.
